## Goal
#### To analyze the probability of being arrested based upon a given age/gender/race combination.

#### - which crimes are committed most frequently by each race?
#### - which crimes occur most frequently in the each respective burough?
#### - what is the trend of overall arrests in nyc, and by year? (then broken down further, trends of arrests based on sex, race, age) Eh.... there's only arrest data records for 2017... so that idea of visualizing the trends over the years is busted.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
arrest_data = pd.read_csv("NYPD_Arrests_Data__Historic_.csv")
arrest_data.head()

In [ ]:
arrest_data.columns

In [ ]:
arrest_data.dtypes

In [ ]:
# use regex in apply to determine valid age_groups
# arrest_data = arrest_data.apply()
import re
valid_age_group_pattern = r'\d+-\d+|<\d+|\d+\+' # | <d+*
re.match(valid_age_group_pattern, "65+")

In [1]:
# Previous attempt which didn't quite work.
# df2 = arrest_data.groupby("AGE_GROUP").filter(lambda x: True if re.match(valid_age_group_pattern, x) else False)
# df3 = arrest_data.filter(lambda x: True if re.match(valid_age_group_pattern, x["AGE_GROUP"]) else False)

arrest_data.dropna(inplace=True)
mask = ~arrest_data["AGE_GROUP"].str.match(r'\d+-\d+|<\d+|\d+\+')

# The following to steps are close... but no cigar.
# Would need to invert the booleans (the - in front of arrest_data does that)
# within the mask Series in order
# to remove the row values which don't match the regex.

# 1. Get names of indexes for which column Age has value 30
indexNames = arrest_data[mask].index
# 2. Delete these row indexes from dataFrame
arrest_data.drop(indexNames, inplace=True)
grouped_by_age = arrest_data.groupby("AGE_GROUP")["AGE_GROUP"]

In [ ]:
arrest_data.columns

In [ ]:
arrest_data.groupby("PERP_SEX")["PERP_SEX"].count()

In [ ]:
grouped_by_race = arrest_data.groupby("PERP_RACE")["PERP_RACE"]

In [ ]:
race_groups_unique = grouped_by_race.unique()
race_groups_count = grouped_by_race.count()

# To facilitate plotting these
display(race_groups_count)
display((race_groups_unique[0], race_groups_count[0]))

In [ ]:
counts = []
race_groups = []
for i in range(len(grouped_by_race.count())):
    race_groups.append(race_groups_unique[i][0])
    counts.append(race_groups_count[i])

display(counts)
display(race_groups)

In [ ]:
sns.barplot(counts, race_groups)

In [ ]:
age_groups_unique = grouped_by_age.unique()
age_groups_count = grouped_by_age.count()

# To facilitate plotting these
display(age_groups_count)
display((age_groups_unique[0], age_groups_count[0]))

In [ ]:
counts = []
age_groups = []
for i in range(len(grouped_by_age.count())):
    age_groups.append(age_groups_unique[i][0])
    counts.append(age_groups_count[i])

display(counts)
display(age_groups)

In [ ]:
sns.barplot(age_groups, counts)

In [ ]:
# Rows w/ these mispellings -> corrected to this
# OFFENSES AGAINST PUBLIC ADMINI -> OFFENSES AGAINST PUBLIC ADMINISTRATION
# MURDER & NON-NEGL. MANSLAUGHTE -> MURDER & NON-NEGL. MANSLAUGHTER
arrest_data["OFNS_DESC"].unique()

In [ ]:
# The result will have keys which correspond to "OFNS_DESC", and accessing that key will provide a DataFrame
# which contains the counts the races and how many instances of this OFNS were perpetrated by that race.
cb = pd.crosstab(arrest_data["PERP_RACE"], arrest_data["OFNS_DESC"], rownames=['race'], colnames=['crimes'])

In [ ]:
# I do feel that visualizing this data which provides very meaningful insight is a challenge...
sns.heatmap(cb)

In [2]:
data_copy1 = arrest_data.copy()

NameError: name 'arrest_data' is not defined

In [3]:
# Am aware .apply isn't the most performance option here.
data_copy1["ARREST_DATE"] = data_copy1["ARREST_DATE"].apply(lambda x: x.split("/")[-1])

NameError: name 'data_copy1' is not defined

In [ ]:
data_copy1["ARREST_DATE"].unique()

In [ ]:
# Why are only 9 of the 18 features being displayed?
# And it doesn't look like it does provide a param to modify that...
# https://seaborn.pydata.org/generated/seaborn.pairplot.html
# So the only option is to create new DataFrames to accomplish that?
sns.pairplot(arrest_data)